In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-aug-2021/train.csv
/kaggle/input/tabular-playground-series-aug-2021/test.csv


In [2]:
df_train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv', index_col='id')
df_test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv',  index_col='id')

In [3]:
df_train.shape

(250000, 101)

In [4]:
int_col= []
for col in df_train.columns:
    if df_train[col].dtype == 'int64':
        int_col.append(col)
int_col

['f1', 'f16', 'f27', 'f55', 'f86', 'loss']

In [5]:
from sklearn.preprocessing import StandardScaler
for col in df_train.columns[:-1]:
    scaler = StandardScaler()
    df_train[col] = scaler.fit_transform(np.array(df_train[col]).reshape(-1,1))
    df_test[col] = scaler.transform(np.array(df_test[col]).reshape(-1,1))

In [6]:
df_train[int_col].corr()

,f1,f16,f27,f55,f86,loss
f1,1.000000,0.005774,-0.000993,-0.001888,0.002761,0.011847
f16,0.005774,1.000000,0.005492,-0.000852,-0.000196,0.009227
f27,-0.000993,0.005492,1.000000,-0.002113,-0.000763,0.003635
f55,-0.001888,-0.000852,-0.002113,1.000000,0.000292,-0.008873
f86,0.002761,-0.000196,-0.000763,0.000292,1.000000,0.000694
loss,0.011847,0.009227,0.003635,-0.008873,0.000694,1.000000


In [7]:
loss_corr = []
for col in df_train.columns:
    loss_corr.append(np.abs(df_train.loss.corr(df_train[col])))
    
df_corr = pd.DataFrame(loss_corr , index=df_train.columns, columns=['corr']).sort_values("corr", ascending=False)

In [8]:
important_feature = set(df_corr.iloc[1:80].index).difference(int_col)

In [9]:
df_train.dtypes.value_counts()

float64    100
int64        1
dtype: int64

In [10]:
df_train.columns

Index(['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9',
       ...
       'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'loss'],
      dtype='object', length=101)

In [11]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [12]:
# df_train = df_train.iloc[:100000]

In [13]:
X_train, X_val, y_train, y_val = train_test_split(df_train.drop(["loss"], axis=1),df_train['loss'],
                                                  test_size=0.20, random_state=42)

In [14]:
X_train = X_train[important_feature]
X_val = X_val[important_feature]
X_test = df_test[important_feature]

In [15]:
params = {'n_estimators': 100, 'max_depth': 7, 'random_state': 0, 
          'min_samples_leaf' : 10, 'learning_rate': 0.1, 'subsample': 0.75, 'loss': 'ls'}
reg = GradientBoostingRegressor()
model=RandomForestRegressor(random_state=0)
xgb = XGBRegressor(n_estimators=100, max_depth=5, eta=0.1, subsample=0.7, colsample_bytree=0.8)

In [16]:
# reg.fit(X_train, y_train)
# np.sqrt(mean_squared_error(y_val, [int(i) for i in reg.predict(X_val)]))

In [17]:
# np.sqrt(mean_squared_error(y_train, [int(i) for i in reg.predict(X_train)]))

In [18]:
xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_val)

np.sqrt(mean_squared_error(y_val, [int(abs(i)) for i in y_pred]))

7.864716142366487

In [19]:
np.sqrt(mean_squared_error(y_train, [int(abs(i)) for i in xgb.predict(X_train)]))

7.760650745910422

In [20]:
output = [int(abs(i)) for i in xgb.predict(X_test)]

In [21]:
output_csv = {"id":[*range(250000,250000+len(output))], "loss":output}
Y_pre = pd.DataFrame(output_csv).astype("int64")
Y_pre.set_index("id", drop=True, append=False, inplace=True)
Y_pre.to_csv("/kaggle/working/submission.csv")

### Thanks For Read my NoteBook :)